https://drive.google.com/open?id=1OletxmPYNkz2ltOr9pyT0b0iBtUWxslh)
Here it is of the format:
word \t label\n
for instance:
postural	B
hypotension	I

here B-> Begin entity, I-> inside entity and O-> outside entity



In [14]:
#Import all required libraries
import nltk
from nltk.corpus import stopwords
import spacy
import random
import time
import numpy as np
import sys
from spacy import displacy
from itertools import chain
import matplotlib.pyplot as plt 
from matplotlib.ticker import MaxNLocator

In [22]:
sent=input("Enter the sentence")
tok_sent=nltk.word_tokenize(sent)
print(tok_sent)

Enter the sentenceToday weather is cold
['Today', 'weather', 'is', 'cold']


In [23]:
sent2=[]
act_sent=[]
stop_words=stopwords.words('english')
for word in tok_sent:
    sent2.append(word.lower())
print(sent2)
for i in sent2:
    if (i not in stop_words):
        act_sent.append(i)
print(act_sent)

['today', 'weather', 'is', 'cold']
['today', 'weather', 'cold']


In [24]:
fin_sent=""
for i in act_sent:
    fin_sent=fin_sent+" "+i
print(fin_sent)

 today weather cold


We have to convert tsv file to the format accepted by spaCy for training.
One of the format supported by spaCy is:
TRAIN_DATA = [[(Sentence, {entities: [(start, end, label)]], ...]

In [18]:
def load_data_spacy(file_path):
    ''' Converts data from:
    word \t label \n word \t label \n \n word \t label
    to: sentence, {entities : [(start, end, label), (stard, end, label)]}
    '''
    file = open(file_path, 'r')
    training_data, entities, sentence, unique_labels = [], [], [], []
    current_annotation = None
    start =0
    end = 0 # initialize counter to keep track of start and end characters
    for line in file:
        line = line.strip("\n").split("\t")
        # lines with len > 1 are words
        if len(line) > 1:
            label = line[1]
            if(label != 'O'):
                label = line[1]+"_Disease"     # the .txt is formatted: label \t word, label[0:2] = label_type
            #label_type = line[0][0] # beginning of annotations - "B", intermediate - "I"
            word = line[0]
            sentence.append(word)
            start = end
            end += (len(word) + 1)  # length of the word + trailing space
           
            if label == 'I_Disease' :  # if at the end of an annotation
                entities.append(( start,end-1, label))  # append the annotation
                              
            if label == 'B_Disease':                         # if beginning new annotation
                entities.append(( start,end-1, label))# start annotation at beginning of word
                
           
           
            if label != 'O' and label not in unique_labels:
                unique_labels.append(label)
 
        # lines with len == 1 are breaks between sentences
        if len(line) == 1:
            if(len(entities) > 0):
                sentence = " ".join(sentence)
                training_data.append([sentence, {'entities' : entities}])
            # reset the counters and temporary lists
            end = 0 
            start = 0
            entities, sentence = [], []
            
    file.close()
    return training_data, unique_labels   

Let us convert our train data,test data and validation data to spaCy format

In [19]:
TRAIN_DATA, LABELS = load_data_spacy(r"C:\Users\Akash UN\Desktop\CustomeNERspaCyv3.0-main\train.tsv")
print(TRAIN_DATA)
print(len(TRAIN_DATA))
TEST_DATA, _ = load_data_spacy(r"C:\Users\Akash UN\Desktop\CustomeNERspaCyv3.0-main\test.tsv")
print(len(TEST_DATA))
VALID_DATA, _ = load_data_spacy(r"C:\Users\Akash UN\Desktop\CustomeNERspaCyv3.0-main\train_dev.tsv")
print(len(VALID_DATA))


[["Selegiline - induced postural hypotension in Parkinson ' s disease : a longitudinal study on the effects of drug withdrawal .", {'entities': [(21, 29, 'B_Disease'), (30, 41, 'I_Disease'), (45, 54, 'B_Disease'), (55, 56, 'I_Disease'), (57, 58, 'I_Disease'), (59, 66, 'I_Disease')]}], ["OBJECTIVES : The United Kingdom Parkinson ' s Disease Research Group ( UKPDRG ) trial found an increased mortality in patients with Parkinson ' s disease ( PD ) randomized to receive 10 mg selegiline per day and L - dopa compared with those taking L - dopa alone .", {'entities': [(32, 41, 'B_Disease'), (42, 43, 'I_Disease'), (44, 45, 'I_Disease'), (46, 53, 'I_Disease'), (132, 141, 'B_Disease'), (142, 143, 'I_Disease'), (144, 145, 'I_Disease'), (146, 153, 'I_Disease'), (156, 158, 'B_Disease')]}], ['Recently , we found that therapy with selegiline and L - dopa was associated with selective systolic orthostatic hypotension which was abolished by withdrawal of selegiline .', {'entities': [(92, 100, 'B_Disea

2842
5385


https://stackoverflow.com/questions/67407433/using-spacy-3-0-to-convert-data-from-old-spacy-v2-format-to-the-brand-new-spacy  The below method is used to convert the train and validation data from old format to new format

In [20]:
import pandas as pd
from tqdm import tqdm
import spacy
from spacy.tokens import DocBin
nlp = spacy.blank("en") # load a new spacy model
db = DocBin() # create a DocBin object

for text, annot in tqdm(TRAIN_DATA): # data in previous format
    doc = nlp.make_doc(text) # create doc object from text
    ents = []
    for start, end, label in annot["entities"]: # add character indexes
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents # label the text with the ents
    db.add(doc)

db.to_disk("./train.spacy") # save the docbin object

db = DocBin()
for text, annot in tqdm(VALID_DATA): # data in previous format
    doc = nlp.make_doc(text) # create doc object from text
    ents = []
    for start, end, label in annot["entities"]: # add character indexes
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents # label the text with the ents
    db.add(doc)

db.to_disk("./valid.spacy") # save the docbin object

100%|█████████████████████████████████████████████████████████████████████████████| 5385/5385 [00:09<00:00, 582.79it/s]


In [7]:
!python -m spacy init fill-config base_config.cfg config.cfg

[+] Auto-filled config with all values

2022-03-16 10:43:35.114409: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-03-16 10:43:35.114493: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.



[+] Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


Train using CLI

In [9]:
 !python -m spacy train config.cfg --verbose --output ./ner_demo/training/ --paths.train train.spacy --paths.dev valid.spacy

[+] Created output directory: ner_demo\training
[i] Saving to output directory: ner_demo\training
[i] Using CPU

2022-03-16 10:46:35.441932: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-03-16 10:46:35.442017: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
[2022-03-16 10:46:44,684] [DEBUG] Config overrides from CLI: ['paths.train', 'paths.dev']
[2022-03-16 10:46:45,959] [INFO] Set up nlp object from config
[2022-03-16 10:46:45,972] [DEBUG] Loading corpus from path: valid.spacy
[2022-03-16 10:46:45,972] [DEBUG] Loading corpus from path: train.spacy
[2022-03-16 10:46:45,972] [INFO] Pipeline: ['transformer', 'ner']
[2022-03-16 10:46:45,982] [INFO] Created vocabulary
[2022-03-16 10:46:45,982] [INFO] Finished initializing nlp object

Downloading:   0%|          | 0.00/478M [00:00<?, ?B/s]
Downloading:   0%|          | 37.0k/478M [00:00<31:42, 263kB/s]




=========================== Initializing pipeline ===========================
[+] Initialized pipeline

============================= Training pipeline =============================
[i] Pipeline: ['transformer', 'ner']
[i] Initial learn rate: 0.0
E    #       LOSS TRANS...  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  -------------  --------  ------  ------  ------  ------
  0       0        5645.48    369.28    0.17    0.20    0.15    0.00
  4     200      123412.72  43124.11   88.37   89.56   87.22    0.88
  8     400        2488.55   4593.25   91.19   89.58   92.86    0.91
 12     600         711.14   1288.54   91.88   94.43   89.47    0.92
[!] Aborting and saving the final best model. Encountered exception:
RuntimeError('[enforce fail at
C:\\actions-runner\\_work\\pytorch\\pytorch\\builder\\windows\\pytorch\\c10\\core\\impl\\alloc_cpu.cpp:81]
data. DefaultCPUAllocator: not enough memory: you tried to allocate 7618560
bytes.')


Downloading:   0%|          | 127k/478M [00:00<25:10, 332kB/s] 
Downloading:   0%|          | 212k/478M [00:00<20:47, 402kB/s]
Downloading:   0%|          | 287k/478M [00:00<19:11, 435kB/s]
Downloading:   0%|          | 388k/478M [00:00<16:11, 516kB/s]
Downloading:   0%|          | 463k/478M [00:00<14:35, 572kB/s]
Downloading:   0%|          | 544k/478M [00:00<13:28, 619kB/s]
Downloading:   0%|          | 624k/478M [00:00<12:42, 657kB/s]
Downloading:   0%|          | 696k/478M [00:01<12:18, 678kB/s]
Downloading:   0%|          | 777k/478M [00:01<12:07, 688kB/s]
Downloading:   0%|          | 857k/478M [00:01<11:51, 704kB/s]
Downloading:   0%|          | 943k/478M [00:01<12:44, 654kB/s]
Downloading:   0%|          | 1.03M/478M [00:01<11:27, 727kB/s]
Downloading:   0%|          | 1.11M/478M [00:01<11:19, 736kB/s]
Downloading:   0%|          | 1.19M/478M [00:01<11:17, 738kB/s]
Downloading:   0%|          | 1.26M/478M [00:01<11:15, 740kB/s]
Downloading:   0%|          | 1.34M/478M [00:02<12

Downloading:  17%|#6        | 80.6M/478M [01:56<09:31, 729kB/s]
Downloading:  17%|#6        | 80.7M/478M [01:56<09:24, 738kB/s]
Downloading:  17%|#6        | 80.7M/478M [01:56<09:23, 739kB/s]
Downloading:  17%|#6        | 80.8M/478M [01:57<09:34, 724kB/s]
Downloading:  17%|#6        | 80.9M/478M [01:57<09:32, 727kB/s]
Downloading:  17%|#6        | 81.0M/478M [01:57<08:55, 778kB/s]
Downloading:  17%|#6        | 81.1M/478M [01:57<10:35, 654kB/s]
Downloading:  17%|#6        | 81.2M/478M [01:57<09:32, 727kB/s]
Downloading:  17%|#6        | 81.2M/478M [01:57<09:22, 739kB/s]
Downloading:  17%|#7        | 81.3M/478M [01:57<09:20, 742kB/s]
Downloading:  17%|#7        | 81.4M/478M [01:57<09:16, 748kB/s]
Downloading:  17%|#7        | 81.5M/478M [01:58<09:15, 749kB/s]
Downloading:  17%|#7        | 81.6M/478M [01:58<14:12, 488kB/s]
Downloading:  17%|#7        | 81.8M/478M [01:58<10:58, 631kB/s]
Downloading:  17%|#7        | 81.9M/478M [01:58<11:23, 608kB/s]
Downloading:  17%|#7        | 82.0M/478M

...

Downloading:  66%|######6   | 318M/478M [08:09<04:00, 700kB/s]
Downloading:  66%|######6   | 318M/478M [08:09<03:51, 724kB/s]
Downloading:  67%|######6   | 318M/478M [08:09<03:51, 724kB/s]
Downloading:  67%|######6   | 318M/478M [08:09<03:47, 737kB/s]
Downloading:  67%|######6   | 318M/478M [08:09<04:32, 615kB/s]
Downloading:  67%|######6   | 318M/478M [08:10<06:19, 442kB/s]
Downloading:  67%|######6   | 318M/478M [08:10<05:30, 507kB/s]
Downloading:  67%|######6   | 318M/478M [08:10<04:59, 559kB/s]
Downloading:  67%|######6   | 318M/478M [08:10<04:33, 612kB/s]
Downloading:  67%|######6   | 318M/478M [08:10<04:22, 636kB/s]
Downloading:  67%|######6   | 318M/478M [08:10<04:13, 660kB/s]
Downloading:  67%|######6   | 319M/478M [08:10<04:03, 685kB/s]
Downloading:  67%|######6   | 319M/478M [08:10<03:54, 712kB/s]
Downloading:  67%|######6   | 319M/478M [08:11<03:56, 706kB/s]
Downloading:  67%|######6   | 319M/478M [08:11<03:45, 740kB/s]
Downloading:  67%|######6   | 319M/478M [08:11<03:51, 7

Downloading:  75%|#######4  | 356M/478M [09:05<02:45, 772kB/s]
Downloading:  75%|#######4  | 356M/478M [09:05<02:49, 751kB/s]
Downloading:  75%|#######4  | 356M/478M [09:06<02:46, 766kB/s]
Downloading:  75%|#######4  | 356M/478M [09:06<02:47, 761kB/s]
Downloading:  75%|#######4  | 357M/478M [09:06<02:52, 739kB/s]
Downloading:  75%|#######4  | 357M/478M [09:06<03:04, 690kB/s]
Downloading:  75%|#######4  | 357M/478M [09:06<03:05, 687kB/s]
Downloading:  75%|#######4  | 357M/478M [09:06<02:57, 716kB/s]
Downloading:  75%|#######4  | 357M/478M [09:06<02:53, 732kB/s]
Downloading:  75%|#######4  | 357M/478M [09:06<02:55, 722kB/s]
Downloading:  75%|#######4  | 357M/478M [09:07<03:20, 632kB/s]
Downloading:  75%|#######4  | 357M/478M [09:07<02:55, 723kB/s]
Downloading:  75%|#######4  | 357M/478M [09:07<02:53, 730kB/s]
Downloading:  75%|#######4  | 357M/478M [09:07<02:52, 734kB/s]
Downloading:  75%|#######4  | 357M/478M [09:07<02:54, 727kB/s]
Downloading:  75%|#######4  | 357M/478M [09:07<02:52, 7

Downloading:  83%|########2 | 396M/478M [10:04<01:57, 734kB/s]
Downloading:  83%|########2 | 396M/478M [10:04<01:57, 733kB/s]
Downloading:  83%|########2 | 396M/478M [10:05<01:57, 734kB/s]
Downloading:  83%|########2 | 396M/478M [10:05<01:56, 737kB/s]
Downloading:  83%|########2 | 396M/478M [10:05<02:02, 701kB/s]
Downloading:  83%|########2 | 396M/478M [10:05<01:57, 732kB/s]
Downloading:  83%|########2 | 396M/478M [10:05<01:57, 732kB/s]
Downloading:  83%|########2 | 396M/478M [10:05<01:56, 740kB/s]
Downloading:  83%|########2 | 396M/478M [10:05<01:55, 741kB/s]
Downloading:  83%|########2 | 396M/478M [10:05<01:57, 730kB/s]
Downloading:  83%|########2 | 396M/478M [10:05<01:56, 736kB/s]
Downloading:  83%|########2 | 396M/478M [10:06<01:57, 726kB/s]
Downloading:  83%|########2 | 396M/478M [10:06<01:57, 728kB/s]
Downloading:  83%|########2 | 397M/478M [10:06<01:54, 745kB/s]
Downloading:  83%|########2 | 397M/478M [10:06<01:56, 732kB/s]
Downloading:  83%|########3 | 397M/478M [10:06<01:55, 7

Downloading:  91%|######### | 434M/478M [11:04<01:03, 729kB/s]
Downloading:  91%|######### | 434M/478M [11:05<01:05, 707kB/s]
Downloading:  91%|######### | 434M/478M [11:05<01:04, 713kB/s]
Downloading:  91%|######### | 434M/478M [11:05<01:10, 650kB/s]
Downloading:  91%|######### | 434M/478M [11:05<01:11, 641kB/s]
Downloading:  91%|######### | 434M/478M [11:05<01:12, 629kB/s]
Downloading:  91%|######### | 434M/478M [11:05<01:12, 626kB/s]
Downloading:  91%|######### | 435M/478M [11:05<01:07, 674kB/s]
Downloading:  91%|######### | 435M/478M [11:05<01:07, 676kB/s]
Downloading:  91%|######### | 435M/478M [11:06<01:05, 693kB/s]
Downloading:  91%|######### | 435M/478M [11:06<01:03, 711kB/s]
Downloading:  91%|######### | 435M/478M [11:06<01:04, 697kB/s]
Downloading:  91%|######### | 435M/478M [11:06<01:03, 707kB/s]
Downloading:  91%|#########1| 435M/478M [11:06<01:05, 687kB/s]
Downloading:  91%|#########1| 435M/478M [11:06<01:05, 687kB/s]
Downloading:  91%|#########1| 435M/478M [11:06<01:06, 6

Downloading:  99%|#########9| 474M/478M [12:04<00:05, 743kB/s]
Downloading:  99%|#########9| 474M/478M [12:04<00:06, 704kB/s]
Downloading:  99%|#########9| 474M/478M [12:04<00:06, 651kB/s]
Downloading:  99%|#########9| 474M/478M [12:04<00:06, 679kB/s]
Downloading:  99%|#########9| 474M/478M [12:04<00:05, 772kB/s]
Downloading:  99%|#########9| 474M/478M [12:04<00:05, 745kB/s]
Downloading:  99%|#########9| 474M/478M [12:04<00:05, 755kB/s]
Downloading:  99%|#########9| 474M/478M [12:04<00:04, 761kB/s]
Downloading:  99%|#########9| 474M/478M [12:05<00:04, 752kB/s]
Downloading:  99%|#########9| 475M/478M [12:05<00:04, 752kB/s]
Downloading:  99%|#########9| 475M/478M [12:05<00:04, 745kB/s]
Downloading:  99%|#########9| 475M/478M [12:05<00:04, 739kB/s]
Downloading:  99%|#########9| 475M/478M [12:05<00:05, 602kB/s]
Downloading:  99%|#########9| 475M/478M [12:05<00:04, 720kB/s]
Downloading:  99%|#########9| 475M/478M [12:05<00:04, 734kB/s]
Downloading:  99%|#########9| 475M/478M [12:05<00:04, 7

Lets test our model on  test data

In [ ]:
ner = spacy.load(R"ner_demo/training/model-best") #load the best model



test_sentences = [x[0] for x in TEST_DATA[0:4000]] # extract the sentences from [sentence, entity]
for x in test_sentences:
    doc = ner(x)
    for ent in doc.ents:
        print(ent.text, ent.start_char, ent.end_char, ent.label_)
    displacy.render(doc,jupyter=True, style = "ent")
   

C:\ProgramData\Anaconda3\lib\site-packages\torch\autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


de 8 10 I_Disease
pointes 11 18 I_Disease
ventricular 19 30 I_Disease
tachycardia 31 42 I_Disease
dilated 111 118 B_Disease
cardiomyopathy 119 133 I_Disease
heart 149 154 B_Disease
failure 155 162 I_Disease


heart 79 84 B_Disease
failure 85 92 I_Disease
cardiomyopathy 114 128 I_Disease
ventricular 156 167 B_Disease
arrhythmias 168 179 I_Disease
ventricular 233 244 B_Disease
tachycardia 245 256 I_Disease


ventricular 34 45 B_Disease
tachycardia 46 57 I_Disease
arrhythmias 138 149 B_Disease


dyspnea 109 116 B_Disease
loss 119 123 B_Disease
of 124 126 I_Disease
consciousness 127 140 I_Disease
macro 155 160 B_Disease
- 161 162 I_Disease
papular 163 170 I_Disease
rash 171 175 I_Disease
pain 268 272 B_Disease


...

lung 119 123 B_Disease
cancer 124 130 I_Disease
cancer 255 261 B_Disease


cancer 38 44 B_Disease
tumour 85 91 B_Disease


cancer 158 164 B_Disease


cancer 179 185 B_Disease


lung 31 35 B_Disease
cancer 36 42 I_Disease


In [25]:
ner = spacy.load(r"ner_demo/training/model-best") #load the best model
doc = ner(fin_sent)
displacy.render(doc,jupyter=True, style = "ent")